In [1]:
# Install the latest version of crossdeploy
! pip install --upgrade crossdeploy

# Node is installed by default in Watson Studio, JupyterLab and JupyterNotebook environments. 
# However, in case your environment does not have node installed, it can be installed with the below command.
# ! pip install "nodejs-bin[cmd]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 MB 2.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 kB 45.7 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 2.12.0
    Uninstalling typeguard-2.12.0:
      Successfully uninstalled typeguard-2.12.0


# Example 1: Understanding how declarative approach works

There are 3 parts to do this example,

- Part A: Running the same code repeatly

- Part B: Modifying the model name

- Part C: Modifying the model's hyperparameter

# Imports

In [2]:
import pandas as pd

from crossdeploy.utils import utils
from crossdeploy.crossdeploy import CrossDeploy

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer

# Define variables

In [3]:
CLOUD_API_KEY = "xxx"
CLOUD_URL = "https://us-south.ml.cloud.ibm.com"
PROJECT_ID = "fc513094-926a-404b-9001-d15a03884123"
SPACE_ID = "9dd079b6-b503-4c77-ae3a-d10bf6d8adae"

# Import data and preprocessing step

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/crossdeploy-io/crossdeploy-examples/main/data/mortgage-default.csv")

label_column = "MortgageDefault"
y = df[label_column]
X = df.drop(label_column, axis=1)

ct = make_column_transformer(
    (OneHotEncoder(), make_column_selector(dtype_include=object)),
    remainder="passthrough"
)

# Part A: Running the same code repeatedly

Running the code below repeatedly, will result in one model being created in your project.

Note that the model id, always remains the same.

In [5]:
MODEL_NAME = "mortgage-model-rf"

model = Pipeline(steps=[
    ("transform", ct), 
    ("clf", RandomForestClassifier(n_estimators=3, max_depth=3, random_state=12345))
])
model.fit(X, y)
model.score(X, y)

flow = CrossDeploy()

flow.ibm.Provider(url=CLOUD_URL, api_key=CLOUD_API_KEY)

model_rf = flow.ibm.Model(model)(
    id = "mortgage-model-rf",
    name = MODEL_NAME,
    project_id = PROJECT_ID,
)

flow.apply()

model_state = utils.get_resource_by_name(model_rf.friendly_unique_id)

print()
print(f"Model ID: {model_state['id']}")

Applying ...

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # ibmcpd_model.crossdeploy_mortgagemodelrf_D0A9BA80 will be created
  + resource "ibmcpd_model" "crossdeploy_mortgagemodelrf_D0A9BA80" {
      + checksum      = "dfef315f632960684a95b017c32ea912"
      + id            = (known after apply)
      + model_path    = "dfef315f632960684a95b017c32ea912.tar.gz"
      + name          = "mortgage-model-rf"
      + project_id    = "fc513094-926a-404b-9001-d15a03884123"
      + software_spec = "runtime-22.2-py3.10"
      + type          = "scikit-learn_1.1"
    }

Plan: 1 to add, 0 to change, 0 to destroy.
ibmcpd_model.crossdeploy_mortgagemodelrf_D0A9BA80: Creating...
ibmcpd_model.crossdeploy_mortgagemodelrf_D0A9BA80: Creation complete after 6s [id=a5bc40f6-7ee7-4535-9c82-491420ce47df]

Apply complete! Resources: 1 added, 0 changed, 0 d

# Part B: Modifying the model name

Try modifying the `MODEL_NAME` and running the code.

You should see that it only results in an update instead of recreating a model.

In addition, the model id remains the same as the earlier example.

In [6]:
MODEL_NAME = "mortgage-model"

model = Pipeline(steps=[
    ("transform", ct), 
    ("clf", RandomForestClassifier(n_estimators=3, max_depth=3, random_state=12345))
])
model.fit(X, y)
model.score(X, y)

flow = CrossDeploy()

flow.ibm.Provider(url=CLOUD_URL, api_key=CLOUD_API_KEY)

model_rf = flow.ibm.Model(model)(
    id = "mortgage-model-rf",
    name = MODEL_NAME,
    project_id = PROJECT_ID,
)

flow.apply()

model_state = utils.get_resource_by_name(model_rf.friendly_unique_id)

print()
print(f"Model ID:   {model_state['id']}")
print(f"Model Name: {model_state['name']}")

Applying ...
ibmcpd_model.crossdeploy_mortgagemodelrf_D0A9BA80: Refreshing state... [id=a5bc40f6-7ee7-4535-9c82-491420ce47df]

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  ~ update in-place

Terraform will perform the following actions:

  # ibmcpd_model.crossdeploy_mortgagemodelrf_D0A9BA80 will be updated in-place
  ~ resource "ibmcpd_model" "crossdeploy_mortgagemodelrf_D0A9BA80" {
        id            = "a5bc40f6-7ee7-4535-9c82-491420ce47df"
      ~ name          = "mortgage-model-rf" -> "mortgage-model"
        # (5 unchanged attributes hidden)
    }

Plan: 0 to add, 1 to change, 0 to destroy.
ibmcpd_model.crossdeploy_mortgagemodelrf_D0A9BA80: Modifying... [id=a5bc40f6-7ee7-4535-9c82-491420ce47df]
ibmcpd_model.crossdeploy_mortgagemodelrf_D0A9BA80: Modifications complete after 1s [id=a5bc40f6-7ee7-4535-9c82-491420ce47df]

Apply complete! Resources: 0 added, 1 changed, 0 destroyed.

Model 

# Part C: Modifying the model's hyperparameter

Change`n_estimators` or `max_depth` to another value.

This is basically a new version of the model as training the model with new hyperparameters is a significant change.

In this case, you should see that the model is destroyed and recreated. A new model id will be produced.

In [7]:
MODEL_NAME = "mortgage-model-rf"

model = Pipeline(steps=[
    ("transform", ct), 
    ("clf", RandomForestClassifier(n_estimators=3, max_depth=10, random_state=12345))
])
model.fit(X, y)
model.score(X, y)

flow = CrossDeploy()

flow.ibm.Provider(url=CLOUD_URL, api_key=CLOUD_API_KEY)

model_rf = flow.ibm.Model(model)(
    id = "mortgage-model-rf",
    name = MODEL_NAME,
    project_id = PROJECT_ID,
)

flow.apply()

model_state = utils.get_resource_by_name(model_rf.friendly_unique_id)

print()
print(f"Model ID: {model_state['id']}")

Applying ...
ibmcpd_model.crossdeploy_mortgagemodelrf_D0A9BA80: Refreshing state... [id=a5bc40f6-7ee7-4535-9c82-491420ce47df]

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
+/- create replacement and then destroy

Terraform will perform the following actions:

  # ibmcpd_model.crossdeploy_mortgagemodelrf_D0A9BA80 must be replaced
+/- resource "ibmcpd_model" "crossdeploy_mortgagemodelrf_D0A9BA80" {
      ~ checksum      = "dfef315f632960684a95b017c32ea912" -> "d8c46fd2ee29d65c1edde2bd0af873e5" # forces replacement
      ~ id            = "a5bc40f6-7ee7-4535-9c82-491420ce47df" -> (known after apply)
      ~ model_path    = "dfef315f632960684a95b017c32ea912.tar.gz" -> "d8c46fd2ee29d65c1edde2bd0af873e5.tar.gz" # forces replacement
      ~ name          = "mortgage-model" -> "mortgage-model-rf"
        # (3 unchanged attributes hidden)
    }

Plan: 1 to add, 0 to change, 1 to destroy.
ibmcpd_model.c

# Clean up

In [9]:
# flow.destroy()

Destroying ...
ibmcpd_model.crossdeploy_mortgagemodelrf_D0A9BA80: Refreshing state... [id=26a1df53-c154-44fa-8d0e-7f567ce52eed]

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # ibmcpd_model.crossdeploy_mortgagemodelrf_D0A9BA80 will be destroyed
  - resource "ibmcpd_model" "crossdeploy_mortgagemodelrf_D0A9BA80" {
      - checksum      = "d8c46fd2ee29d65c1edde2bd0af873e5" -> null
      - id            = "26a1df53-c154-44fa-8d0e-7f567ce52eed" -> null
      - model_path    = "d8c46fd2ee29d65c1edde2bd0af873e5.tar.gz" -> null
      - name          = "mortgage-model-rf" -> null
      - project_id    = "fc513094-926a-404b-9001-d15a03884123" -> null
      - software_spec = "runtime-22.2-py3.10" -> null
      - type          = "scikit-learn_1.1" -> null
    }

Plan: 0 to add, 0 to change, 1 to destroy.
ibmcpd_model.crossdeploy_mortgagemodelrf_